In [11]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
from adafactor import AdaFactor
# Training settings

class ProtoArgs(object):
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 10
        self.cuda = 3



args = ProtoArgs()

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)


kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = Net()
if args.cuda:
    model.cuda(args.cuda)

# optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = AdaFactor(model.parameters(), non_constant_decay = True, enable_factorization=False)

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda:
            data, target = data.cuda(args.cuda), target.cuda(args.cuda)
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args.cuda:
            data, target = data.cuda(args.cuda), target.cuda(args.cuda)
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, args.epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.399207
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.006243
Train Epoch: 1 [1280/60000 (2%)]	Loss: 1.480791
Train Epoch: 1 [1920/60000 (3%)]	Loss: 1.479675
Train Epoch: 1 [2560/60000 (4%)]	Loss: 1.049796
Train Epoch: 1 [3200/60000 (5%)]	Loss: 1.516404
Train Epoch: 1 [3840/60000 (6%)]	Loss: 1.120633
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.415526
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.087480
Train Epoch: 1 [5760/60000 (10%)]	Loss: 0.998421
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.075759
Train Epoch: 1 [7040/60000 (12%)]	Loss: 0.941075
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.617702
Train Epoch: 1 [8320/60000 (14%)]	Loss: 0.716373
Train Epoch: 1 [8960/60000 (15%)]	Loss: 0.984068
Train Epoch: 1 [9600/60000 (16%)]	Loss: 0.857867
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.832931
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.484417
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.884776
Train Epoch: 1 [12160/60000 (20%)]	Loss: 0.909070
Train Epoch: 1 [12800/60000 (

Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.873443
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.386720
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.512651
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.430896
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.665589
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.456695
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.464724
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.341708
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.486958
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.565826
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.239854
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.528244
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.423471
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.418721
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.399247
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.430618
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.784110
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.641979
Train Epoch: 2 [56960/60000 (95%)]	Loss: 0.473250
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.354063


Process Process-27:


Train Epoch: 3 [28800/60000 (48%)]	Loss: 0.389400


KeyboardInterrupt


KeyboardInterrupt: 

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 42, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 115, in default_collate
    elif isinstance(batch[0], collections.Mapping):
  File "/opt/conda/lib/python3.6/abc.py", line 182, in __instancecheck__
    if subclass in cls._abc_cache:
